# Integrating Big Data Analytics and Convolutional Neural Networks for Pest and Disease Detection and Classification

In [1]:
#Install the necessary libraries
#!pip install torch
#!pip install torchvision
#!pip install opencv-python

In [2]:
#Import all the necessary libraries
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
import os
from PIL import Image
import numpy as np
import shutil

In [3]:
#List the contents of the root directory in HDFS
!hdfs dfs -ls /

Found 4 items
drwxr-xr-x   - hduser supergroup          0 2024-03-20 15:45 /crop_pest_disease_data
drwxr-xr-x   - hduser supergroup          0 2024-03-07 10:06 /output1
drwxr-xr-x   - hduser supergroup          0 2024-03-07 13:14 /output2
drwxr-xr-x   - hduser supergroup          0 2024-03-14 14:03 /user1


In [4]:
#List the contents of the crop pest and disease data directory in HDFS
!hdfs dfs -ls /crop_pest_disease_data

Found 4 items
drwxr-xr-x   - hduser supergroup          0 2024-03-20 12:05 /crop_pest_disease_data/Cashew
drwxr-xr-x   - hduser supergroup          0 2024-03-20 16:27 /crop_pest_disease_data/Cassava
drwxr-xr-x   - hduser supergroup          0 2024-03-20 18:00 /crop_pest_disease_data/Maize
drwxr-xr-x   - hduser supergroup          0 2024-03-20 13:04 /crop_pest_disease_data/Tomato


In [5]:
#List the contents of Cashew in HDFS
!hdfs dfs -ls /crop_pest_disease_data/Cashew

Found 2 items
drwxr-xr-x   - hduser supergroup          0 2024-03-20 12:02 /crop_pest_disease_data/Cashew/test_set
drwxr-xr-x   - hduser supergroup          0 2024-03-20 12:48 /crop_pest_disease_data/Cashew/train_set


In [6]:
#List the contents of Cassava in HDFS
!hdfs dfs -ls /crop_pest_disease_data/Cassava

Found 2 items
drwxr-xr-x   - hduser supergroup          0 2024-03-20 17:55 /crop_pest_disease_data/Cassava/test_set
drwxr-xr-x   - hduser supergroup          0 2024-03-20 17:12 /crop_pest_disease_data/Cassava/train_set


In [7]:
#List the contents of Maize in HDFS
!hdfs dfs -ls /crop_pest_disease_data/Maize

Found 2 items
drwxr-xr-x   - hduser supergroup          0 2024-03-20 13:58 /crop_pest_disease_data/Maize/test_set
drwxr-xr-x   - hduser supergroup          0 2024-03-20 18:35 /crop_pest_disease_data/Maize/train_set


In [8]:
#List the contents of Tomato in HDFS
!hdfs dfs -ls /crop_pest_disease_data/Tomato

Found 2 items
drwxr-xr-x   - hduser supergroup          0 2024-03-20 12:59 /crop_pest_disease_data/Tomato/test_set
drwxr-xr-x   - hduser supergroup          0 2024-03-20 13:30 /crop_pest_disease_data/Tomato/train_set


## Reading data from hadoop

In [24]:
crops_classes = {
    "Cashew": ["anthracnose3102", "gumosis1714", "healthy5877", "leaf miner3466", "red rust4751"],
    "Cassava": ["bacterial blight3241", "bacterial_blight", "brown_spot", "green_mite", "healthy","mosaic"],
    "Maize": ["fall armyworm", "grasshoper", "healthy", "leaf beetle", "leaf blight", "leaf_spot","streak virus"],
    "Tomato": ["healthy", "leaf blight", "leaf curl", "septoria leaf spot","verticulium wilt"]
}

train_data_rdds = []

for crop, classes in crops_classes.items():
    crop_train_data_rdd = []
    for class_name in classes:
        # Ensure proper concatenation of HDFS URI and class name

        hdfs_path = f"hdfs://localhost:9000/crop_pest_disease_data/{crop}/train_set/{class_name}"
        train_images_rdd = sc.binaryFiles(hdfs_path)
        crop_train_data_rdd.append(train_images_rdd)
    train_data_rdds.append(crop_train_data_rdd)

In [25]:
type(train_data_rdds)

list

In [26]:
from pyspark.ml.image import ImageSchema
ImageSchema.readImages(f"hdfs://localhost:9000/crop_pest_disease_data")

AttributeError: '_ImageSchema' object has no attribute 'readImages'

In [ ]:
spark.read.format("image").load(f"hdfs://localhost:9000/crop_pest_disease_data")

In [23]:
# Iterate over each element in the list train_data_rdds
for element in train_data_rdds:
  print_element(element)  # This will print or process each element


[org.apache.spark.api.java.JavaPairRDD@583a0574, org.apache.spark.api.java.JavaPairRDD@230e0f8d, org.apache.spark.api.java.JavaPairRDD@6e726d34, org.apache.spark.api.java.JavaPairRDD@2e244d48, org.apache.spark.api.java.JavaPairRDD@26bdec5d]
[org.apache.spark.api.java.JavaPairRDD@759cfd60, org.apache.spark.api.java.JavaPairRDD@87a252a, org.apache.spark.api.java.JavaPairRDD@4b87760, org.apache.spark.api.java.JavaPairRDD@4daeacae, org.apache.spark.api.java.JavaPairRDD@5f4b3f86, org.apache.spark.api.java.JavaPairRDD@6411230]
[org.apache.spark.api.java.JavaPairRDD@418566ef, org.apache.spark.api.java.JavaPairRDD@79f669ad, org.apache.spark.api.java.JavaPairRDD@7f47de1a, org.apache.spark.api.java.JavaPairRDD@71999350, org.apache.spark.api.java.JavaPairRDD@51b09846, org.apache.spark.api.java.JavaPairRDD@10462b64, org.apache.spark.api.java.JavaPairRDD@2df8392f]
[org.apache.spark.api.java.JavaPairRDD@3f27c0a3, org.apache.spark.api.java.JavaPairRDD@6a801da9, org.apache.spark.api.java.JavaPairRDD@4

In [20]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType

# Assuming each element in train_data_rdds has 5 elements (adjust if different)
schema = StructType([StructField(str(i), StringType(), True) for i in range(5)])

# Create DataFrame with the adjusted schema
df = sqlContext.createDataFrame(train_data_rdds, schema)


ValueError: Length of object (6) does not match with length of fields (5)

In [ ]:
# Initialize lists to store features and labels
features_list = []
labels_list = []

# Iterate over each RDD
for crop_rdd_list in train_data_rdds:
    for rdd in crop_rdd_list:
        # Extract crop name from RDD path
        crop = rdd.name().split('/')[-3]
        # Extract class name from RDD path
        class_name = rdd.name().split('/')[-1]
        # Extract features (images) and labels (class names) from file paths
        features_with_paths = rdd.collect()
        if features_with_paths:  # Check if RDD is not empty
            features = [features for _, features in features_with_paths]
            labels = [class_name] * len(features)
            # Append features and labels to the lists
            features_list.extend(features)
            labels_list.extend(labels)

# Now, features_list contains all the images and labels_list contains corresponding class names


In [ ]:
#sc master - running  locally
sc.master

In [ ]:
#Initialize Spark session
spark = SparkSession.builder \
    .appName("Crop Pest Disease Detection") \
    .getOrCreate()
warnings.filterwarnings("ignore")

In [ ]:
def load_images_from_directory(directory):
    images = []
    labels = []
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path) and os.listdir(class_path):  # Check if directory exists and not empty
            for filename in os.listdir(class_path):
                # Process image files here
                # Example: load image using OpenCV or any other library
                # image = cv2.imread(os.path.join(class_path, filename))
                # images.append(image)
                images.append(os.path.join(class_path, filename))  # Append image file path
                labels.append(class_name)
        else:
            print(f"Warning: Directory {class_path} is empty or does not exist.")
    return images, labels

crops_classes = {
    "Cashew": ["anthracnose", "gumosis", "healthy", "leaf miner", "red rust"],
    "Cassava": ["bacterial_blight", "brown_spot", "green_mite", "healthy", "mosaic"],
    "Maize": ["fall armyworm", "grasshopper", "healthy", "leaf beetle", "leaf blight", "leaf_spot", "streak virus"],
    "Tomato": ["healthy", "leaf blight", "leaf curl", "septoria leaf spot", "verticillium wilt"]
}

data_directory = "hdfs://localhost:9000/crop_pest_disease_data"

test_data_rdds = []

for crop, classes in crops_classes.items():
    crop_test_data_rdd = []
    for class_name in classes:
        hdfs_path = f"{data_directory}/{crop}/test_set/{class_name}"
        images, labels = load_images_from_directory(hdfs_path)
        if images and labels:  # Check if images and labels are not empty
            crop_test_data_rdd.append((images, labels))
    test_data_rdds.append(crop_test_data_rdd)


In [ ]:
# Hadoop directory path containing image files
hadoop_dir_path = "hdfs://localhost:9000/crop_pest_disease_data"

# Read image files from Hadoop directory
image_df = spark.read.format("image").load(hadoop_dir_path)


In [ ]:
# Show a snippet of the DataFrame
image_df.show(5)

In [ ]:
# View DataFrame schema
image_df.printSchema()

In [ ]:
crop_df = datasets.ImageFolder(root=hadoop_data_path, transform=transform)

## Perform Image processing


The dataset has already been processed. Here is the image preprocessing that was done to the data.
- All images were captured, separated,and saved in their respective folders according to the plant type.
- The images were annotated and labelled
- Image Cropping and size reduction

## Data Processing and Cleaning

### Checking the image sizes of each crop

In [ ]:
import os
from PIL import Image

# Assuming you have already defined and loaded your dataset
# dataset = datasets.ImageFolder(root=hadoop_data_path, transform=transform)

# Get the list of classes (crops) in the dataset
classes = dataset.classes

# Create a dictionary to store image sizes for each crop
crop_image_sizes = {crop: [] for crop in classes}

# Iterate through the dataset
for image_path, label in dataset.samples:
    # Open the image using PIL
    image = Image.open(image_path)
    # Get the size of the image
    width, height = image.size
    # Get the crop name using the label
    crop_name = classes[label]
    # Append the image size to the corresponding crop in the dictionary
    crop_image_sizes[crop_name].append((width, height))

# Print the image sizes for each crop
for crop, sizes in crop_image_sizes.items():
    print(f"Crop: {crop}")
    print(f"Total images: {len(sizes)}")
    print(f"Average image size: {sum([w for w, h in sizes]) / len(sizes)} x {sum([h for w, h in sizes]) / len(sizes)}")
    print()

### check if the pixel value have been normalized to a range suitable for training neural networks


In [ ]:
import torch
from torchvision import datasets, transforms

# Assuming you have already defined and loaded your dataset
# dataset = datasets.ImageFolder(root=hadoop_data_path, transform=transform)

# Define a transformation to convert PIL images to PyTorch tensors
transform = transforms.ToTensor()

# Load a few sample images from the dataset
sample_loader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=True)
for images, labels in sample_loader:
    # Convert images to numpy arrays and print pixel value range
    for image in images:
        # Check pixel value range
        min_pixel_value = torch.min(image)
        max_pixel_value = torch.max(image)
        print(f"Min pixel value: {min_pixel_value}, Max pixel value: {max_pixel_value}")

import os
import torch
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

# Define transformations for preprocessing
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load image data using PyTorch
hadoop_data_path = "hdfs://path/to/crop_pest_disease_data"
dataset = datasets.ImageFolder(root=hadoop_data_path, transform=transform)

# Split data into training and testing datasets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders for batch processing
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)


## Check if there are any defective images

In [ ]:
data_dir='hdfs://localhost:9000/crop_pest_disease_data'
bad_img_list=[]
total=0
good=0
bad=0
classes=sorted(os.listdir(data_dir))
for klass in classes:
    good_class=0
    bad_class=0
    total_class=0
    msg=f'processing class {klass}'
    print(msg, '\r', end= '')
    classpath=os.path.join(data_dir, klass)
    flist=sorted(os.listdir(classpath))
    for f in flist:
        total +=1
        total_class +=1
        fpath=os.path.join(classpath,f)
        try:
            img= Image.open(fpath) 
            array=np.asarray(img)
            good +=1
            good_class +=1
        except:
            bad_img_list.append(fpath)
            bad +=1
            bad_class +=1
    
    msg=f'class {klass} contains {total_class} files, {good_class} are valid image files and {bad_class} defective image files'
    print (msg)
msg=f'the dataset contains {total} image files, {good} are valid image files and {bad} are defective image files'
print (msg)
if bad>0:
    ans=input('to print a list of defective image files enter P, to not print press Enter')
    if ans == 'P' or ans == 'p':
        for f in bad_img_list:
            print(f)

In [ ]:
Create a corrected dataset with the defective image files removed

In [ ]:
# you can use this dataset to create a model.
working_dir=r'/kaggle/working/'
corrected_dir=os.path.join(working_dir, 'corrected dataset') # where the corrected dataset will be stored
copied_count = 0
if os.path.isdir(corrected_dir):
    shutil.rmtree(corrected_dir) # make sure the corrected_dir is empty
os.mkdir(corrected_dir)
for klass in classes:
    classpath=os.path.join(data_dir, klass)
    dest_classpath=os.path.join(corrected_dir, klass)
    os.mkdir(dest_classpath)
    flist= os.listdir(classpath)
    for f in flist:
        fpath=os.path.join(classpath,f)
        dest_fpath=os.path.join(dest_classpath,f)
        if fpath not in bad_img_list:
            shutil.copy(fpath, dest_fpath)
            copied_count +=1
msg=f'{copied_count} valid image files were stored in {corrected_dir}'
print(msg)

In [ ]:
#Split the training data into training and validation datasets

### Training Data

## Validation Data

### Test Data

In [1]:
crops_classes = {
    "Cashew": ["anthracnose", "gumosis", "healthy", "leaf miner", "red rust"],
    "Cassava": ["bacterial_blight", "brown_spot", "green_mite", "healthy","mosaic"],
    "Maize": ["fall armyworm", "grasshoper", "healthy", "leaf beetle", "leaf blight", "leaf_spot","streak virus"],
    "Tomato": ["healthy", "leaf blight", "leaf curl", "septoria leaf spot","verticulium wilt"]
}

test_data_rdds = []

for crop, classes in crops_classes.items():
    crop_test_data_rdd = []
    for class_name in classes:


        hdfs_path = f"hdfs://localhost:9000/crop_pest_disease_data/{crop}/test_set/{class_name}"
        test_images_rdd = spark.sparkContext.binaryFiles(hdfs_path)
        crop_test_data_rdd.append(test_images_rdd)
    test_data_rdds.append(crop_test_data_rdd)

In [7]:
hdfs_path = f"hdfs://localhost:9000/crop_pest_disease_data/Maize/test_set/grasshopper/0maize_valid_grasshoper.JPG"
test_images = spark.sparkContext.binaryFiles(hdfs_path)

In [13]:
type(test_images)

pyspark.rdd.RDD

In [14]:
data=test_images.collect()
print(data)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapreduce.lib.input.InvalidInputException: Input path does not exist: hdfs://localhost:9000/crop_pest_disease_data/Maize/test_set/grasshopper/0maize_valid_grasshoper.JPG
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:340)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.listStatus(FileInputFormat.java:279)
	at org.apache.spark.input.StreamFileInputFormat.setMinPartitions(PortableDataStream.scala:51)
	at org.apache.spark.rdd.BinaryFileRDD.getPartitions(BinaryFileRDD.scala:51)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2328)
	at org.apache.spark.rdd.RDD.$anonfun$collect$1(RDD.scala:1019)
	at org.apache.spark.rdd.RDDOperat

In [ ]:
# spark is from the previous example.
sc = spark.sparkContext

# Path to your image data
cropdata = "file:///home/hduser/Big data and advanced analytics data"

# Read image files into a DataFrame
crop_dataframe = spark.read.format("image").load(cropdata)

# Show the DataFrame schema and first few rows
crop_dataframe.printSchema()
crop_dataframe.show()

# Stop SparkSession
spark.stop()

In [ ]:
!git add .
!git commit -m "comment"
!git push origin main

In [ ]:
file = sc.textFile

In [ ]:
#Import all libraries required for EDA, Preprocessing, Model building, Model Testing, Model EValuation and Visualization
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from keras.models import Sequential
from tensorflow.compat.v1.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D

In [ ]:
#Default size for input images
width=256
height=256
depth=3epoch_ = 25
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
root_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/'
INIT_LR = 1e-3

In [ ]:
#Convert Image into NUmPy array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
#Image and Lable List
image_list, label_list = [], []

In [ ]:
try:

In [ ]:
import PIL

In [ ]:
print("Pillow Version:", PIL.__version__)

In [ ]:
#Load all the images in a directory
from os import listdir
from matplotlib import image
#Load all images i